# Data modelling on Bike Sharing data in London

Data aquired from: https://s3.amazonaws.com/capitalbikeshare-data/index.html  
Data project initiated: 25/01/2019  
Author: Sedar Olmez

Data modelling:  
    GDS1: Data Gathering, Preparation and Exploration.  
    GDS2: Data Representation and Transformation.  
    GDS3: Computing with Data.   
    GDS4: Data Visualisation and Presentation.   
    GDS5: Data Modelling.   
    GDS6: Science about Data Science. 

Assessment:
![Assessment](assessment.png)

# Dataset information
I have two datasets, one I will be using to clean and the other to perform machine learning algorithms on. The reason for why I am using two is because the original dataset I want to use has already been cleaned, therefore, I need to perform some cleaning tasks on a smaller dataset with a similar theme.

1) Dataset (cleaning): Walking_Cycling.csv - London Cycling % and Walking % by Local Authority `COLUMNS: 'LA code', 'Local Authority', 'Year', 'Frequency', 'Walking_%', 'Cycling_%'`


2) Dataset (Machine Learning/visualisation): capitalbikeshare-tripdata-washington.csv - Capital Bike sharing information in Washington DC 2017, large dataset with a lot of useful data. `COLUMNS: 'Duration'	'Start Date'	'End Date'	'Start Station Number'	'Start Station Name'	'End Station Number'	'End Station Name'	'Bike Number'	'Member Type'`

In [2]:
# Libraries
from __future__ import print_function
import matplotlib.pyplot as plt
import seaborn as sea
import pandas as pd
import numpy as np
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor as DTR

## Small dataset - Walking_Cycling.csv

In [3]:
%%time
# Load up the journeys.csv dataset into dataframes
dataframe_journeys = pd.read_csv('data/Walking-Cycling.csv')
dataframe_journeys.columns = ['LA code', 'Local Authority', 'Year', 'Frequency', 'Walking_%', 'Cycling_%']
dataframe_journeys.dropna()

CPU times: user 8.21 ms, sys: 4.39 ms, total: 12.6 ms
Wall time: 16.6 ms


In [4]:
dataframe_journeys.head()

,LA code,Local Authority,Year,Frequency,Walking_%,Cycling_%
0,E09000001,City of London,2010/11,1x per month,78,30
1,E09000002,Barking and Dagenham,2010/11,1x per month,60,8
2,E09000003,Barnet,2010/11,1x per month,65,10
3,E09000004,Bexley,2010/11,1x per month,65,11
4,E09000005,Brent,2010/11,1x per month,62,14


In [5]:
list(dataframe_journeys)

['LA code', 'Local Authority', 'Year', 'Frequency', 'Walking_%', 'Cycling_%']

In [6]:
# We will now convert the Local Authority column to string
dataframe_journeys['Local Authority'] = dataframe_journeys['Local Authority'].astype('|S')

In [7]:
dataframe_journeys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 6 columns):
LA code            1204 non-null object
Local Authority    1204 non-null object
Year               1204 non-null object
Frequency          1204 non-null object
Walking_%          1204 non-null int64
Cycling_%          1204 non-null object
dtypes: int64(1), object(5)
memory usage: 56.5+ KB


#### In Pandas, dtype(obj) == python.dtype(str) therefore we set the string to the maximum bytes of the longest string stored by using

```.astype(|S)```

In [8]:
# We must now convert both the Walking_% and Cycling_% to float as this would make regression more accurate.
# However we must first convert Cycling_% from String to Int then Float.
dataframe_journeys['Cycling_%'].astype(int)

ValueError: invalid literal for long() with base 10: '-'

In [9]:
# An invalit literal for long() error is thrown, this means there are values in the column which cannot be converted to int
# Let us identify these columns
dataframe_journeys['Cycling_%'] = pd.to_numeric(dataframe_journeys['Cycling_%'], errors='coerce')

In [10]:
# We found the row with the problem, for row 326 the column Cycling_% had '-' so we replaced it with NaN.
print (dataframe_journeys[ pd.to_numeric(dataframe_journeys['Cycling_%'], errors='coerce').isnull()])

       LA code Local Authority     Year    Frequency  Walking_%  Cycling_%
326  E09000026       Redbridge  2011/12  5x per week         17        NaN


In [11]:
# We must replace the NaN to an integer value i.e. 0 so we can produce a pairplot later using seaborn.
dataframe_journeys = dataframe_journeys.fillna(0.0)

In [12]:
# The 'Cycling_%' column was converted to int, now we can focus on changing both columns to floats.
dataframe_journeys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 6 columns):
LA code            1204 non-null object
Local Authority    1204 non-null object
Year               1204 non-null object
Frequency          1204 non-null object
Walking_%          1204 non-null int64
Cycling_%          1204 non-null float64
dtypes: float64(1), int64(1), object(4)
memory usage: 56.5+ KB


In [13]:
# Here we convert Walking_% to float from int
dataframe_journeys['Walking_%'] = dataframe_journeys['Walking_%'].astype(float)

In [14]:
# As can be seen, we converted the columns to floats.
dataframe_journeys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 6 columns):
LA code            1204 non-null object
Local Authority    1204 non-null object
Year               1204 non-null object
Frequency          1204 non-null object
Walking_%          1204 non-null float64
Cycling_%          1204 non-null float64
dtypes: float64(2), object(4)
memory usage: 56.5+ KB


In [15]:
dataframe_journeys.head()

,LA code,Local Authority,Year,Frequency,Walking_%,Cycling_%
0,E09000001,City of London,2010/11,1x per month,78.0,30.0
1,E09000002,Barking and Dagenham,2010/11,1x per month,60.0,8.0
2,E09000003,Barnet,2010/11,1x per month,65.0,10.0
3,E09000004,Bexley,2010/11,1x per month,65.0,11.0
4,E09000005,Brent,2010/11,1x per month,62.0,14.0


## Dataset cleaned and optimised, now to explore we will use our main dataset on the bikeshare data in washington DC for the year 2017 (already cleaned)

In [16]:
%%time
# Load up the journeys.csv dataset into dataframes
dataframe_washington = pd.read_csv('data/2017-capitalbikeshare-tripdata-washington.csv')
dataframe_washington.columns = ['Duration', 'Start Date', 'End Date', 'Start Station Number', 'Start Station Name', 'End Station Number',
                                'End Station Name', 'Bike Number', 'Member Type']
dataframe_washington.dropna()

CPU times: user 2.62 s, sys: 136 ms, total: 2.75 s
Wall time: 1.31 s


In [17]:
dataframe_washington.head()

,Duration,Start Date,End Date,Start Station Number,Start Station Name,End Station Number,End Station Name,Bike Number,Member Type
0,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member
1,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual
2,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual
3,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual
4,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual


In [18]:
dataframe_washington.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646510 entries, 0 to 646509
Data columns (total 9 columns):
Duration                646510 non-null int64
Start Date              646510 non-null object
End Date                646510 non-null object
Start Station Number    646510 non-null int64
Start Station Name      646510 non-null object
End Station Number      646510 non-null int64
End Station Name        646510 non-null object
Bike Number             646510 non-null object
Member Type             646510 non-null object
dtypes: int64(3), object(6)
memory usage: 44.4+ MB


In [19]:
# We can strip the 'W' from the Bike Numbers to convert it into an Integer which we can then add to our visualisation 
# Data manipulation for later.
dataframe_washington['Bike Number'] = dataframe_washington['Bike Number'].str[1:]

In [20]:
# The problematic rows were identified
dataframe_washington['Bike Number'] = pd.to_numeric(dataframe_washington['Bike Number'], errors='coerce')
print (dataframe_washington[ pd.to_numeric(dataframe_washington['Bike Number'], errors='coerce').isnull()])

        Duration           Start Date             End Date  \
51884       4756  2017-01-12 09:11:13  2017-01-12 10:30:29   
52358       1872  2017-01-12 10:36:32  2017-01-12 11:07:45   
130357       490  2017-01-25 13:07:42  2017-01-25 13:15:52   
145330      7077  2017-01-27 08:30:55  2017-01-27 10:28:53   
218827      2529  2017-02-07 10:32:38  2017-02-07 11:14:47   
228772      2470  2017-02-08 09:52:07  2017-02-08 10:33:18   
268213     77023  2017-02-14 11:14:44  2017-02-15 08:38:28   
275600      2374  2017-02-15 10:02:10  2017-02-15 10:41:44   
283135      3490  2017-02-16 12:05:23  2017-02-16 13:03:34   
347404      2097  2017-02-23 08:12:49  2017-02-23 08:47:46   
348846      9067  2017-02-23 09:38:55  2017-02-23 12:10:03   
515949      3107  2017-03-18 10:24:11  2017-03-18 11:15:58   
551769       298  2017-03-22 10:58:34  2017-03-22 11:03:32   
643328      5164  2017-03-31 08:59:10  2017-03-31 10:25:14   

        Start Station Number  \
51884                  31082   
52358

In [21]:
# Filled all the NaN rows with 0 for Bike Number.
dataframe_washington = dataframe_washington.fillna(0)

In [22]:
# I now converted the Bike Number column to integer so we can add it to our data analysis.
dataframe_washington['Bike Number'] = dataframe_washington['Bike Number'].astype(int)

In [23]:
dataframe_washington.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646510 entries, 0 to 646509
Data columns (total 9 columns):
Duration                646510 non-null int64
Start Date              646510 non-null object
End Date                646510 non-null object
Start Station Number    646510 non-null int64
Start Station Name      646510 non-null object
End Station Number      646510 non-null int64
End Station Name        646510 non-null object
Bike Number             646510 non-null int64
Member Type             646510 non-null object
dtypes: int64(4), object(5)
memory usage: 44.4+ MB


## Lets run some queries

In [24]:
# Lets have a look at all the Lincoln Memorial Start Station journeys.
dataframe_washington.loc[dataframe_washington['Start Station Name']=='Lincoln Memorial', :]

,Duration,Start Date,End Date,Start Station Number,Start Station Name,End Station Number,End Station Name,Bike Number,Member Type
1,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,894,Casual
4,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,22786,Casual
5,1603,2017-01-01 00:08:11,2017-01-01 00:34:55,31258,Lincoln Memorial,31270,8th & D St NW,20890,Casual
24,36943,2017-01-01 00:25:42,2017-01-01 10:41:26,31258,Lincoln Memorial,31286,11th & O St NW,20716,Casual
25,1903,2017-01-01 00:25:43,2017-01-01 00:57:27,31258,Lincoln Memorial,31281,8th & O St NW,367,Casual
294,1040,2017-01-01 08:22:12,2017-01-01 08:39:32,31258,Lincoln Memorial,31283,14th & L St NW,765,Member
297,1606,2017-01-01 08:25:26,2017-01-01 08:52:12,31258,Lincoln Memorial,31222,New York Ave & 15th St NW,21808,Member
301,1527,2017-01-01 08:31:57,2017-01-01 08:57:24,31258,Lincoln Memorial,31641,2nd St & Massachusetts Ave NE,20507,Member
539,2717,2017-01-01 10:37:26,2017-01-01 11:22:43,31258,Lincoln Memorial,31258,Lincoln Memorial,1339,Casual
554,2504,2017-01-01 10:40:31,2017-01-01 11:22:16,31258,Lincoln Memorial,31258,Lincoln Memorial,1271,Casual


In [25]:
## Has anyone started at Lincoln Memorial and Ended up at Maine Ave & 7th St SW?
dataframe_washington.loc[(dataframe_washington['Start Station Name'] =='Lincoln Memorial') & 
                         (dataframe_washington['End Station Name'] == 'Maine Ave & 7th St SW'), :]

,Duration,Start Date,End Date,Start Station Number,Start Station Name,End Station Number,End Station Name,Bike Number,Member Type
68066,1388,2017-01-14 15:35:35,2017-01-14 15:58:44,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,21733,Member
70739,2765,2017-01-15 13:30:17,2017-01-15 14:16:23,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,20844,Casual
106130,907,2017-01-21 09:58:11,2017-01-21 10:13:18,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,21898,Member
106144,809,2017-01-21 09:59:58,2017-01-21 10:13:27,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,21755,Member
153169,2303,2017-01-28 14:01:30,2017-01-28 14:39:53,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,21004,Member
253858,987,2017-02-11 20:13:03,2017-02-11 20:29:31,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,23163,Member
272053,763,2017-02-14 19:09:49,2017-02-14 19:22:32,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,22536,Member
315527,2115,2017-02-19 17:58:32,2017-02-19 18:33:47,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,21714,Casual
315547,2059,2017-02-19 17:59:09,2017-02-19 18:33:29,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,1353,Casual
315549,2059,2017-02-19 17:59:10,2017-02-19 18:33:29,31258,Lincoln Memorial,31609,Maine Ave & 7th St SW,20040,Casual


In [26]:
## Was the bike number 185 used?
for i in range(dataframe_washington['Bike Number'].size):
    if(dataframe_washington['Bike Number'].iloc[i] == 185):
        print('Bike 185 used!')

Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 used!
Bike 185 u

In [27]:
## 185 has been used quite a lot actually, let us now write a method which counts the number of times something appears.
def counter(column, object_name):
    counter_index = 0
    for i in range(dataframe_washington[column].size):
        if(dataframe_washington[column].iloc[i] == object_name):
            counter_index += 1
    return counter_index

In [28]:
# As we can see, in 2017 the bike 185 was used 214 times.
print(counter('Bike Number', 185))

214


In [29]:
# Let's have a look at how many times members commuted and compare that to the number of times 
# casual people commuted with bikes.
print(counter('Member Type', 'Member'))

528434


In [30]:
# And now how many times casual users used the bikes in Washington:
print(counter('Member Type', 'Casual'))

118076


## More members have used the bikes during 2017 than casual users. Almost 5 times the number of casual users. 

1. We will use the [Capital Bikeshare API](https://secure.capitalbikeshare.com/map/) to visualise a docking station and see if we can pinpoint the furthest distance travelled from that station.
2. We will try to deduce how many people ended their journey at a docking station which is the closest to a museum. 

## The first docking station we will concentrate on is `Constitution Ave & 2nd St NW/DOL`
<img src="Images\Const_ave.png" width="600" height="500">

In [35]:
# Now lets define a method which will tell us if a value exists in a column.
def data_identification(column_name, value):
    for i in range (dataframe_washington[column_name].size):
        if (dataframe_washington[column_name].iloc[i] == value):
            return ("Value Exists!")
            break

In [40]:
# We check to see if the station number 31271 does exist in our dataset, that way we know the API and dataset are callibrated
print(data_identification('Start Station Number', 31271))

Value Exists!


In [65]:
#Now we want to see the furthest distance travelled from the station 31271. We first assign the rows with the station
#start number = 31271 to the variable list_of_stations.
list_of_stations = dataframe_washington.loc[dataframe_washington['Start Station Number'] == 31271, :]
#We now run the max() function on the `Duration` column to return the maximum duration
list_of_stations['Duration'].max()

81885

## The maximum duration of travel from the starting station `Constitution Ave & 2nd St NW/DOL` was `01` hour, `15` minutes and `14` seconds. The `end station` was the same as start station and the user was a `member`. 

In [67]:
list_of_stations.loc[list_of_stations['Duration'] == 81885]

,Duration,Start Date,End Date,Start Station Number,Start Station Name,End Station Number,End Station Name,Bike Number,Member Type
228091,81885,2017-02-08 08:46:57,2017-02-09 07:31:43,31271,Constitution Ave & 2nd St NW/DOL,31271,Constitution Ave & 2nd St NW/DOL,20197,Member
